In [ ]:
from typing import TypedDict, Optional
from langchain_aws import ChatBedrock
from langgraph.graph import StateGraph, END
import boto3
import json

# ==== AWS Configuration ====
REGION = "eu-west-2"
TARGET_LAMBDA = "test_function_26May2025"
KNOWLEDGE_BASE_ID = "ZTPJ2E7RTK"
MODEL_ARN = "arn:aws:bedrock:eu-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"

lambda_client = boto3.client("lambda", region_name=REGION)
brt_client = boto3.client("bedrock-agent-runtime", region_name=REGION)

# ==== LangGraph-compatible functions (nodes) ====

def lambda_summary_node(state):
    question = state["question"]
    schema_id = state.get("schema_id", "vitic_lhasa")

    response = lambda_client.invoke(
        FunctionName=TARGET_LAMBDA,
        InvocationType='RequestResponse',
        Payload=json.dumps({
            "question": question,
            "schema_id": schema_id
        })
    )
    payload = json.loads(response['Payload'].read())

    return {
        **state,  # preserve all previous state
        "narrative_response": payload.get("response", "")
    }

def bedrock_rag_node(state):
    question = state["question"]
    narrative_response = state["narrative_response"]

    combined_input = (
        f"[Structured Database Response]\n{narrative_response}\n\n"
        f"[Follow-up Question]\n{question}"
    )

    rag_response = brt_client.retrieve_and_generate(
        input={"text": combined_input},
        retrieveAndGenerateConfiguration={
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                "knowledgeBaseId": KNOWLEDGE_BASE_ID,
                "modelArn": MODEL_ARN
            }
        }
    )
    return {"final_answer": rag_response["output"]["text"]}

def reflector_node(state):
    question = state["question"]
    narrative_response = state["narrative_response"]

    prompt = f"""
    You are an expert agent. Given the following database response and user question, determine whether the answer is sufficient.

    [Question]
    {question}

    [Narrative Response]
    {narrative_response}

    Respond only as:
    Verdict: COMPLETED or INCOMPLETE
    Justification: (brief explanation)
    """

    model = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", region_name=REGION)
    result = model.invoke(prompt)
    message = result.content

    verdict = "COMPLETED" if "COMPLETED" in message.upper() else "INCOMPLETE"
    justification = message.split("Justification:")[-1].strip() if "Justification:" in message else ""
    # ➕ Increment retry count
    retry_count = state.get("retry_count", 0)
    retry_count += 1

    # 🖨️ Print iteration and summary
    print(f"\n--- Reflector Iteration #{retry_count} ---")
    print(f"Verdict: {verdict}")
    print(f"Justification: {justification}\n")

    return {
        **state,
        "verdict": verdict,
        "justification": justification,
        "retry_count": retry_count
    }
    
def router(state):
    if state.get("verdict") == "COMPLETED":
        return "bedrock_rag"
    else:
        return "lambda_summary"  # Retry the summary step

# ==== Define the graph ====
class AgentState(TypedDict, total=False):
    question: str
    schema_id: Optional[str]
    narrative_response: Optional[str]
    final_answer: Optional[str]
    retry_count: int
    verdict: Optional[str]
    justification: Optional[str]
    
graph = StateGraph(AgentState)
graph.add_node("lambda_summary", lambda_summary_node)
graph.add_node("bedrock_rag", bedrock_rag_node)
graph.add_node("reflector", reflector_node)
graph.add_conditional_edges("reflector", router)

# Define transitions

graph.set_entry_point("lambda_summary")
graph.add_edge("lambda_summary", "reflector")
# reflector decides between rerunning lambda_summary or proceeding to bedrock_rag
graph.set_finish_point("bedrock_rag")

# Compile the graph
app = graph.compile()


In [ ]:
inputs = {
    "question": "Is Omeprazole carcinogenic?",
    "schema_id": "vitic_lhasa"
}

result = app.invoke(inputs)
print("Final RAG response:\n", result["final_answer"])

--- Reflector Iteration #1 ---
Verdict: INCOMPLETE
Justification: While the response provides information on carcinogenicity studies in rats and mice, it does not directly answer whether Omeprazole is carcinogenic in humans. The relevance of the animal findings to human cancer risk is stated as uncertain due to the lack of data from human studies.


--- Reflector Iteration #2 ---
Verdict: COMPLETED
Justification: The response provides a clear overview of the available evidence on the potential carcinogenicity of Omeprazole, including study details in rats and mice, the specific tumor types observed, and a conclusion that synthesizes the findings across species. While there is no direct human data, the response acknowledges this limitation and suggests that the positive findings in animal studies may have relevance to humans. Overall, the response seems to sufficiently address the original question regarding whether Omeprazole is carcinogenic.

Final RAG response:
 Based on the information provided in the search results, there is evidence that Omeprazole may have carcinogenic potential, particularly in causing gastric carcinoid tumors in rats after chronic exposure. However, the carcinogenic effects appear to be species-specific, with no significant increase in tumor incidence observed in mice under the tested conditions. The search results indicate that in a 2-year study in rats, Omeprazole caused a statistically significant increase in malignant gastric carcinoid tumors, with higher potency in females (source 1). However, in an 18-month study in mice, no statistically significant increase in gastric carcinoid tumors was observed (source 1). While there is no direct data on carcinogenicity in humans provided in the search results, some sources suggest that the carcinogenic effects observed in rats may be relevant to humans due to the implicated mechanisms of action (source 3, source 5). However, other sources indicate that the findings in rats may not translate to humans due to species differences in physiology and metabolism (source 3, source 4). In summary, while Omeprazole has shown carcinogenic potential in rats, particularly in causing gastric carcinoid tumors, the evidence is not conclusive for carcinogenicity in humans due to species differences and lack of direct human data. Further research and clinical data may be needed to fully assess the carcinogenic risk of Omeprazole in humans.

In [ ]:
inputs = {
    "question": "Is tiazofurin mutagenic?",
    "schema_id": "vitic_lhasa"
}

result = app.invoke(inputs)
print("Final RAG response:\n", result["final_answer"])

--- Reflector Iteration #1 ---
Verdict: INCOMPLETE
Justification: The response provides evidence that tiazofurin induces chromosomal damage and may present a genotoxic hazard based on in vitro micronucleus assays in human cells. However, it acknowledges that additional studies, especially in vivo genotoxicity and carcinogenicity studies, would be needed to further evaluate the potential carcinogenic hazard. Therefore, the response does not provide a definitive answer to whether tiazofurin is mutagenic, and more information is needed.


--- Reflector Iteration #2 ---
Verdict: INCOMPLETE
Justification: The response provides information on the potential genotoxicity and mutagenicity of tiazofurin based on in vitro micronucleus assays, but it does not directly answer whether tiazofurin is mutagenic or not. The conclusion states that the evidence is suggestive of a genotoxic concern but insufficient to conclude on the carcinogenic potential, which implies that there is some evidence of mutagenicity but not a definitive conclusion.


--- Reflector Iteration #3 ---
Verdict: INCOMPLETE
Justification: The response provides details on the mutagenic potential of tiazofurin based on in vitro micronucleus assays, but does not directly answer whether the compound is mutagenic or not. The conclusion states that the evidence is suggestive of a genotoxic concern, but insufficient to conclude on the carcinogenic potential, leaving the question of whether tiazofurin is mutagenic unanswered.


--- Reflector Iteration #4 ---
Verdict: INCOMPLETE
Justification: The response provides information on the potential mutagenic effects of tiazofurin based on two in vitro micronucleus assays, but it does not directly answer whether tiazofurin is mutagenic or not. The conclusion states that additional studies, especially in vivo genotoxicity and carcinogenicity studies, would be needed to further evaluate the potential mutagenic hazard of tiazofurin. Therefore, the information provided is incomplete to definitively determine if tiazofurin is mutagenic.


--- Reflector Iteration #5 ---
Verdict: INCOMPLETE
Justification: While the response provides evidence from two in vitro studies showing that tiazofurin induces chromosomal damage, it lacks information on whether this compound is mutagenic or not. The studies mentioned only assess chromosomal damage, which is one aspect of genotoxicity, but does not directly address mutagenicity. To fully answer whether tiazofurin is mutagenic, additional information from gene mutation assays or other relevant tests specifically evaluating mutagenic potential would be needed.

Max retry count reached — forcing progression to 'bedrock_rag'
Final RAG response:
 The search results do not provide enough information to determine if tiazofurin is mutagenic or not. The information given is about two in vitro micronucleus assays showing tiazofurin induced chromosomal damage in human cells, but these studies had some reliability limitations. No clear conclusion about tiazofurin's mutagenicity can be made from the provided information.